In [21]:
import pickle
import numpy as np

class hidden_neuron():
    def __init__(self, weights, activation):
        self.weights = weights
        self.activation = activation
        self.output_neuron = None

    def __str__(self):
        x = "[{}, {}, {}]" .format(self.weights, self.activation, self.output_neuron)
        return x


class output_neuron():
    def __init__(self, class_):
        self.class_ = class_
    
    def __str__(self):
        x = "[{}]" .format(self.class_)
        return x
        
    def get_class(self):
        return self.class_


class rce():
    def __init__(self, r_max=3):
        self.r_max = r_max
        self.q = None #number of neurons in hidden layer
        self.m = None #number of neurons in output layer
        self.modif = None #identifeir of change in network
        self.p = None #index of vector from training set
        self.hit = None #hit identifier
        self.k = None #index of neuron
        self.hidden_layer = []
        self.output_layer = []


    def __str__(self):
        x = "#############################\n"
        x += "neurons in hidden layer: {}\n" .format(self.q)
        x += "neurons in output layer: {}\n" .format(self.m)
        x += "change in network {}; hit identifier {}\n" .format(self.modif, self.hit)
        x += "index of vector {}; index of neuron {}\n" .format(self.p, self.k)
        x += "hidden layer: "
        for i, item in enumerate(self.hidden_layer):
            x += "{}, " .format(item)
        x += "\n"    
        x += "output layer: "
        for i, item in enumerate(self.output_layer):   
            x += "{}, " .format(item)
        x += "\n"
        x += "#############################\n"

        return x


    def fit(self, x, y):
        self.hidden_layer = []
        self.output_layer = []
        self.q = 0
        self.m = 0

        while True:
            self.modif = False
            self.p = 1

            while True:
                self.hit = False
                self.k = 1
                if self.k <= self.q:
                    while True:

                        u_k = (x[self.p-1][0] - self.hidden_layer[self.k-1].weights[0])**2
                        u_k += (x[self.p-1][1] - self.hidden_layer[self.k-1].weights[1])**2
                        u_k = u_k**(0.5)

                        if u_k > self.hidden_layer[self.k-1].activation:
                            pass
                        elif self.hidden_layer[self.k-1].output_neuron.get_class() == y[self.p-1]:
                            self.hit = True
                        else:
                            self.hidden_layer[self.k-1].activation = u_k / 2
                            self.modif = True

                        self.k += 1
                        if self.k > len(self.hidden_layer):
                            break

                    if not self.hit:
                        self.add_new_neuron(x, y)
                        self.modif = True
                else:
                    self.modif = True
                    self.add_new_neuron(x, y)

                self.p += 1
                if self.p > len(y):
                    break

            if self.modif == False:
                print(self)
                return


    def add_new_neuron(self, x, y):
        hidden_ = hidden_neuron([x[self.p-1][0], x[self.p-1][1]], self.r_max)
        self.hidden_layer.append(hidden_)
        self.q += 1
        self.modif = True
        exists = False

        for i, item in enumerate(self.output_layer):
            if item.get_class() == y[self.p-1]:
                exists = True
                output_ = item
                break

        if not exists:
            output_ = output_neuron(y[self.p-1])
            self.output_layer.append(output_)
            self.m += 1

        hidden_.output_neuron = output_


    def predict(self, x):
        predictions = []
        for i, item in enumerate(x):
            prediction = []
            for e, elem in enumerate(self.hidden_layer):
                u_k = (item[0] - elem.weights[0])**2
                u_k += (item[1] - elem.weights[1])**2
                u_k = u_k**(0.5)
                if u_k <= elem.activation:
                    if elem.output_neuron.get_class() not in prediction:
                        prediction.append(elem.output_neuron.get_class())
            predictions.append(prediction)
        return predictions


    def save_model(self, name):
        """
        method saves model
        """
        with open("{}.pkl" .format(name), "wb") as model:
            pickle.dump(self, model, pickle.HIGHEST_PROTOCOL)

            
    def load_model(self, name):
        """
        method loads model
        """
        with open("{}.pkl".format(name), "rb") as model:
            tmp_model = pickle.load(model)

        self.r_max = tmp_model.r_max
        self.q     = tmp_model.q
        self.m     = tmp_model.m
        self.modif = tmp_model.modif
        self.p     = tmp_model.p
        self.hit   = tmp_model.hit
        self.k     = tmp_model.k
        self.hidden_layer = tmp_model.hidden_layer
        self.output_layer = tmp_model.output_layer

In [22]:
rce = rce()

x = np.array([[2,0], [0,1], [2,1], [0,3]])
y = np.array([0,1,0,1])
rce.fit(x, y)

x = np.array([[0,2], [1,0]])
print(rce.predict(x))

x = np.array([[0,3], [2,2], [0,0], [1,1]])
y = np.array([0,0,1,1])
rce.fit(x, y)

x = np.array([[0,2], [1,0]])
print(rce.predict(x))

#############################
neurons in hidden layer: 3
neurons in output layer: 2
change in network False; hit identifier True
index of vector 5; index of neuron 4
hidden layer: [[2, 0], 1.118033988749895, [0]], [[0, 1], 1.0, [1]], [[0, 3], 1.4142135623730951, [1]], 
output layer: [0], [1], 
#############################

[[1], [0]]
#############################
neurons in hidden layer: 3
neurons in output layer: 2
change in network False; hit identifier True
index of vector 5; index of neuron 4
hidden layer: [[0, 3], 1.5, [0]], [[0, 0], 1.5, [1]], [[2, 2], 0.7071067811865476, [0]], 
output layer: [0], [1], 
#############################

[[0], [1]]
